## Creating Widgets that Navigate through NBA Statistics
By Ashley Radford

Downloading and using nba data from stats.nba.com to create interactive widgets that present plots analyzing the players' shots. This data is used to demonstrate data transformation and visualization via the split-apply-combine approach.

### Data download
The `get_nba_data()` function from the helper_basketball module will be used to download multiple nba datasets that will be needed for the drop down selection widgets. These datasets incude the teams data, player data, and shots data. End visualizations will require specific selected data grabbed from the shots data, which in and of itself will require specific paramaters. Thus, after retrieving our three necessary datasets using the `get_nba_data()` function, we will create another function with three parameters (team id, name id, and game date id). This function will return the necessary parameters for a specifc team, player, and game date for the `get_nba_data()` function used to download shots data. This function, `choose_player_game()`, is tested along with `get_nba_data()` below.

In [1]:
# necessary importations
import numpy as np
import pandas as pd
from ipywidgets import Dropdown, Button
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
from IPython.display import clear_output
import helper_basketball as h

In [2]:
# choose season and year
# note: current_year is 20BB when latest_season is 20AA-BB
current_year = 2017
latest_season = '2016-17'

In [ ]:
# getting teams data (refer to function comments in helper_basketball.py for appropriate parameters)
teams_params = {'LeagueID':'00'}
teams = h.get_nba_data('commonTeamYears', teams_params)
teams.head()

In [ ]:
# link that data is pulled from
# http://stats.nba.com/stats/commonTeamYears?LeagueID=00

In [ ]:
# getting player data
# can put whatever in season, not only current season
players_params = {'LeagueID':'00', 'Season': latest_season, 'IsOnlyCurrentSeason':'0'}
players = h.get_nba_data('commonallplayers', players_params)
players.head()

In [ ]:
# getting shots data
shots_params = {'PlayerID':'0',
                'PlayerPosition':'',
                'Season': latest_season,
                'ContextMeasure':'FGA',
                'DateFrom':'',
                'DateTo':'',
                'GameID':'',
                'GameSegment':'',
                'LastNGames':'0',
                'LeagueID':'00',
                'Location':'',
                'Month':'0',
                'OpponentTeamID':'0',
                'Outcome':'',
                'Period':'0',
                'Position':'',
                'RookieYear':'',
                'SeasonSegment':'',
                'SeasonType':'Regular Season',
                'TeamID':'0',
                'VsConference':'',
                'VsDivision':''}

shotdata = h.get_nba_data('shotchartdetail', shots_params)
shotdata.head()

In [ ]:
# function using the widgets selections
def choose_player_game(teamid, playerid='0', gameid=None):
    if teamid == None: teamid = '0'
    if playerid == None: playerid = '0'
    if gameid == None: gameid = ''
    chosen_params = {'PlayerID':playerid,
                     'PlayerPosition':'',
                     'Season': latest_season,
                     'ContextMeasure':'FGA',
                     'DateFrom':'',
                     'DateTo':'',
                     'GameID':gameid,
                     'GameSegment':'',
                     'LastNGames':'0',
                     'LeagueID':'00',
                     'Location':'',
                     'Month':'0',
                     'OpponentTeamID':'0',
                     'Outcome':'',
                     'Period':'0',
                     'Position':'',
                     'RookieYear':'',
                     'SeasonSegment':'',
                     'SeasonType':'Regular Season',
                     'TeamID':teamid,
                     'VsConference':'',
                     'VsDivision':''}
    return chosen_params

# test function for parameters Golden State Warriors, Stephen Curry, and 10/25/2016 game
# only works when current_year is 2017 and latest_season is 2016-17
player_game = h.get_nba_data('shotchartdetail', choose_player_game('1610612744', '201939', '0021600003'))
player_game.head()

### Subsetting and cleaning data for widget creation
Later we will be creating the 3 widgets that will be used to pull data with the `chose_player_game()` and `get_nba_data()` functions. However, we first want to set up the data types and make sure to subset only the current teams and active players (players who played at least one game in the selected season) for our drop down selections. This is because we plan to have the following widgets: first one to select the team, then one to pick a player from the selected team, and then a third to pick a specific game that that player took shots in. We will do a check of the data and then follow with some more data clean up.

In [ ]:
# setting up series types
teams.ABBREVIATION = teams.ABBREVIATION.astype('category')
teams.TEAM_ID      = teams.TEAM_ID.astype('category')
teams.MIN_YEAR     = teams.MIN_YEAR.astype('int')
teams.MAX_YEAR     = teams.MAX_YEAR.astype('int')

# taking current teams
teams = teams[(teams.MIN_YEAR <= (current_year-1)) & (teams.MAX_YEAR >= current_year)]
# taking current and active players
players = players[players.TEAM_ID.isin(teams.TEAM_ID) & players.PERSON_ID.isin(shotdata.PLAYER_ID)]
# taking active players shotdata
shotdata = shotdata[shotdata.PLAYER_ID.isin(players.PERSON_ID)]

In [ ]:
# checking teams data
teams.head()

In [ ]:
# checking players data
players.head()

In [ ]:
# checking shots 
shotdata.head()

### Adding in some columns for later convenience
Eventually, we are going to want to take the name of a team from the teams dataframe for convenience. Thus, we will create a `TEAM_NAME` column in the 'teams' dataframe. We will do this by using the merging in the `TEAM_NAME` column from the `players` dataframe.

In [ ]:
# creating TEAM_NAME column on teams by merging data from players
team_names = players[['TEAM_ABBREVIATION', 'TEAM_NAME']].copy().drop_duplicates()
teams = pd.merge(teams, team_names, left_on='ABBREVIATION', right_on='TEAM_ABBREVIATION')

# getting rid of TEAM_ABBREVIATION column
teams = teams.drop(['TEAM_ABBREVIATION'], axis=1) # only returns values, so needs to be reassigned
teams.head()

### Creating a useful function for current and later use
In the following code, the function `readable_data()` is created to allow for proper formating of the game dates that are in `shotdata.`

In [ ]:
# creating readable dates function
def readable_date(date):
    mm = date[4:6]
    dd = date[6:]
    yyyy = date[:4]
    readable = mm+'/'+dd+'/'+yyyy
    return readable

# changing the GAME_DATE column in shotdata
temp = []
for date in shotdata.GAME_DATE:
    read_date = readable_date(date)
    temp += [read_date]

shotdata['GAME_DATE'] = temp

### Something to consider
Before continuing, it is important that we understand our data and its source. What does it represent? What do we expect to see in our dataframes? What NBA rules would affect certain numbers and labels? For example, overtime exists in the NBA and so there may be more than 4 quarters represented. We see that there are actually up to 8 represented in our `shotdata` dataframe. No surprises here. However, are players stationary throughout a season, do they stay on the same team? Those familiar with the NBA know of the trading system. Our `shotdata` once again shows this to be true. To look at a specifc example, we see that the player DeMarcus Cousins' `TEAM_NAME` column contains two teams, the Sacramento Kings and the New Orleans Pelicans. Yet, the `Players` dataframe does not support this. In fact, there only exists a row for each unique player and their latest team, so in this case DeMarcus Cousins only seems to be a part of the New Orleans Pelicans. This issue will affect our dropdown menus in that there will be no way to properly pull Cousins' shots data under the Sacramento Kings. We will address this problem by first finding all the players who were in multiple teams throughout the season and then creating a dicitonary with their namre and the previous teams' ids. 

In [ ]:
# getting other teams        
player_other_dd = dict()
for p, data in shotdata.groupby('PLAYER_NAME'):
    all_teams = np.unique(shotdata.loc[shotdata['PLAYER_NAME']==p]['TEAM_ID']) # grab all of their teams (dont need .copy() because multi-dtype)
    if len(all_teams) > 1:
        current_team = shotdata.loc[shotdata['PLAYER_NAME']==p]['TEAM_ID'].values[-1] # grab their current team (ordered by date)
        other_teams = all_teams[np.where(all_teams!=current_team)]                    # only grab previous teams
        player_other_dd[p] = other_teams                                              # create dict keys and values

player_other_dd

We will now create a dataframe from this dictionary. Note that some of these players have more than one previous team. We want a dataframe where each row represents one team. And so we must comb through our new dataframe and ensure this to be the case, adjusting the rows when needed.

In [ ]:
# creating player_other dataframe
player_other = pd.DataFrame({
                 'player': list(player_other_dd.keys()),
                 'previous_team': list(player_other_dd.values())
               })

# seperating the extra games teams (if they had more than 2 teams in a season)
for i, l in enumerate(player_other['previous_team']):
    player_other['previous_team'][i] = l[0]                # taking string from first positioned team
    c=0                                                    # to help with new index count
    if len(l) > 1:                                         # if they have more than 1 other team
        l = (l[1:])                                        # not considering first other team
        for t in l:
            extra = pd.DataFrame({
                      'player': player_other['player'][i], # grabbing player
                      'previous_team': t                   # grabbing all other teams seperately
                    }, index = [len(player_other)+c])      # getting new idexes
            player_other = player_other.append(extra)      # appending the extra dataframe
            c+=1
            
# sorting player_other alphabetically and resetting indices numbers
player_other = player_other.sort_values('player').reset_index(drop=True) 
player_other[17:23]

We will now grab copies of the rows from the `players` dataframe that correspond to the players in `player_other.` We will then adjust that row changing the team specific columns to represent the previous teams that were not accounted for (taking special care of thsoe with more than one previous team). We will be creating a new dataframe from these rows (note the the first line of code).

In [ ]:
# creating other_teams dataframe from player dataframe (to append onto later)
other_teams = pd.DataFrame()
for player in player_other_dd.keys():
    for i in range(len(player_other_dd[player])):
        # grabbing the additional row
        new_row = players[players['DISPLAY_FIRST_LAST']==player].copy()
        index = new_row.index[0] # finding index label
        
        # changing team columns
        team_id = player_other_dd[player][i]
        
        # updating the team columns
        new_row.loc[index,'TEAM_ID'] = team_id
        new_row.loc[index,'TEAM_NAME'] = teams['TEAM_NAME'][np.where(teams['TEAM_ID']==team_id)[0][0]]
        new_row.loc[index,'TEAM_ABBREVIATION'] = teams['ABBREVIATION'][np.where(teams['TEAM_ID']==team_id)[0][0]]
        new_row.loc[index,'TEAM_CITY'] = players['TEAM_CITY'][players.index[np.where(players['TEAM_ID']==team_id)[0][0]]]
        new_row.loc[index,'TEAM_CODE'] = players['TEAM_CODE'][players.index[np.where(players['TEAM_ID']==team_id)[0][0]]]
        
        # adding the updated column
        other_teams = other_teams.append(new_row)
    
other_teams[17:22]

Now that we have a complete dataframe of all previous teams and their corresponding player, we will be appending this to our original `players` dataframe. We now have all players and all of their teams accounted for in our `players` dataframe.

In [ ]:
# appending other_teams onto players dataframe and sorting
players = players.append(other_teams).sort_values('DISPLAY_LAST_COMMA_FIRST')
players.head(9)

### Creating dictonaries for the widget drop down menus
Now that our data is set up and ready to go, three dictionaries will be created from each dataset. We will first create a dictionary containing the team names and mascots as keys, with the teamids as the corresponding values. The second dictionary will have teamids as its keys and dictionaries as its values. These dictonary values will have player names as its keys and the corresponding playerid as its value. Finally, the third dictionary will have playerids as its keys and will have dictionaries as its values too. These dictionary values will have game dates as its keys and the corresponding datids for its values. Later, all the id keys from these three dictonaries will be used to fetch the wanted data using the `choose_player_game()` function.

In [ ]:
# getting dictionaries ready for the widgets
# creating (team: teamid) dictionary
teams = teams.sort_values('ABBREVIATION') # sorting teams by abbreviation
team_dd_text = teams.ABBREVIATION + ', ' + teams.TEAM_NAME
team_dd = dict(zip(team_dd_text, teams.TEAM_ID))

# creating players by team dictionary
# teamid: {player: personid})
players = players.sort_values('DISPLAY_FIRST_LAST') # sorting players by first name
plyr_by_team_dd = dict()
for t, p in players.groupby('TEAM_ID'):
    plyr_by_team_dd[t] = dict(zip(p.DISPLAY_FIRST_LAST, p.PERSON_ID))

# creating games by player dictionary
# (personid, teamid): {game date: gameid}
game_by_player_dd = dict()
for p, g in shotdata.groupby(['PLAYER_ID','TEAM_ID']):
    game_by_player_dd[p] = dict(zip(g.GAME_DATE, g.GAME_ID))

Finally, the interactive widgets will be created. The first will contain all the teams, the second all the players, and the third all the games. They will all act independently from one another as of now.

In [ ]:
# creating widgets
# change selected00 and selected01 based on current_season (see helper_basketball)
selected00 = 'NOP, Pelicans'
selected01 = 'DeMarcus Cousins'

team_menu = Dropdown(options=team_dd, label=selected00)
plyr_menu = Dropdown(options=plyr_by_team_dd[team_dd[selected00]], label=selected01)
game_menu = Dropdown(options=game_by_player_dd[(plyr_by_team_dd[team_dd[selected00]][selected01], team_dd[selected00])])

display(team_menu, plyr_menu, game_menu)

### Downloading data with changing widget states
Now, event handlers such as `options`, `value`, `observe`, and `on_click` will be added to the previous cell in order to accomodate changes in selections (or the lack of changes). The nested games within the players within the teams will no longer act independently. A fetch button will also be added to allow the user to get the id numbers. The `update_team()` and `update_player()` function will update each dropdown menu as the other dropdown menus are changed. The `get_data()` function will then pull the ids from the dictionaries for each click of the button.

In [ ]:
fetch_button = Button(description='Get Data!', icon='check')
display(team_menu, plyr_menu, game_menu, fetch_button)

# update players list
def update_team(change): 
    plyr_menu.options = plyr_by_team_dd[change['new']]
    plyr_menu.value = list(plyr_by_team_dd[change['new']].values())[0]

team_menu.observe(update_team, names='value')

# update game list
def update_player(change):
    if change['new'] != None: # update game from player only when player is changed, ow will do it through above cell
        game_menu.options = game_by_player_dd[(change['new'], team_menu.value)]
        game_menu.value = list(game_by_player_dd[(change['new'], team_menu.value)].values())[0]

plyr_menu.observe(update_player, names='value')

# get data action
# change is a dictionary holding the information about the change
def get_data(change):
    print(team_menu.value, plyr_menu.value, game_menu.value) 
    
fetch_button.on_click(get_data)

### Data transformation and visualization
In the following code, `get_visuals()` will be defined. This function will now be called with the `on_click` handler instead of `get_data()`. To summarize, this function will output two visualizations of the data requested from the dropdown menus. A more in depth description is as follows. First `get_visuals()` will take the ids of the team, player, and date that is selected. Instead of outputting it as before, `get_visuals()` will pass the ids onto `choose_player_game()` which will be passed into `get_nba_data()`. This will be done two seperate times, as each visual requires different information. At this point, data will be retrieved and will be used to create the outputted visualizations. The first visualization will be a bar chart comparing the shots made for each game of the player for the team that is selected (will be grouping by date). The chosen date, as well as its bar, will be highlighted in green. Here, one could examine how the players' shots per game vary as the season goes on. The second visualization will be specific to the date chosen (will be grouping by period). It will plot the percentages of shots made (so it will take into account the shots attempted) at each period (as well as the overtime periods) for the selected game date. Here, one could observe how the player performs as time runs out.

In [ ]:
%matplotlib inline
fetch_button = Button(description='Get Data!', icon='check')
display(team_menu, plyr_menu, game_menu, fetch_button)

def get_visuals(change):
    clear_output()
    display(team_menu, plyr_menu, game_menu, fetch_button)
    
    teamid = str(team_menu.value)
    playerid = str(plyr_menu.value)
    gameid = str(game_menu.value)
    
    # getting data set up for plot of total 
    # shots each game for chosen player
    player_shots = h.get_nba_data('shotchartdetail', choose_player_game(teamid, playerid))
    player_shots_dd = dict()
    for d, s in player_shots.groupby('GAME_DATE'):
        player_shots_dd[d] = sum(s.SHOT_MADE_FLAG)
    
    player_shots_df = pd.DataFrame({
                        'game_date': list(player_shots_dd.keys()),
                        'shots_made': list(player_shots_dd.values()),
                      })
    
    # getting game date (from value) to color later
    gameid_index = list(player_shots['GAME_ID']).index(gameid)
    match_date = list(player_shots['GAME_DATE'])[gameid_index]
    date_location = list(player_shots_df.game_date).index(match_date)
    
    # getting prettier dates
    temp = player_shots_df.game_date.copy()
    for i in range(len(temp)):
        read_date = readable_date(temp[i])
        temp[i] = read_date
    player_shots_df['game_date'] = temp
    
    # plotting total shots each game for chosen player
    figsize(11, 9)
    objects = player_shots_df['game_date']
    y_pos = np.arange(len(objects))
    shots_plot = plt.bar(y_pos, player_shots_df['shots_made'], align = 'center')
    plt.xticks(y_pos, objects, rotation=90, fontsize=8);
    
    plt.xlabel("Game Date")
    plt.ylabel("Shots Made")
    plt.title("Total Shots Made Each Game by " + str(list(player_shots['PLAYER_NAME'])[0]) 
              + " for " + str(list(player_shots['TEAM_NAME'])[0]))
    
    # coloring specific date bar and label
    shots_plot.get_children()[date_location].set_color('g') 
    plt.gca().get_xticklabels()[date_location].set_color('g')
    
    plt.show() # to show both graphs
    
    # getting data set up for percentage of shots
    # made in each period for chosen game    
    period_percentage = h.get_nba_data('shotchartdetail', choose_player_game(teamid, playerid, gameid))
    period_percentage_dd = dict()
    for p, s in period_percentage.groupby('PERIOD'):
        period_percentage_dd[p] = sum(s.SHOT_MADE_FLAG) / sum(s.SHOT_ATTEMPTED_FLAG)
    
    period_percentage_df = pd.DataFrame({
                        'period': sorted(list(period_percentage_dd.keys())),
                        'percentage_made': list(period_percentage_dd.values()),    
                      })
    
    # plotting percentage of shots made in each period for chosen game
    plt.plot((period_percentage_df['period']), period_percentage_df['percentage_made'], '-o')
    plt.xlabel("Period")
    plt.ylabel("Percentage of Shots Made")
    plt.title("Percentage of Shots Made Per Period on " + str(readable_date(match_date)))
    
    ticks=[1,2,3,4]
    ot = [] # for overtime periods
    quarters = list(period_percentage.PERIOD.unique()) # quarters where shots were taken
    for i in quarters: # getting possible overtime periods
        if i > 4:
            ot.append(i)
    if len(ot) != 0:
        ticks += ot
    
    plt.xticks(ticks)
    plt.xlim((0.5,ticks[-1] + 0.5))
    plt.ylim((-0.1,1.1))
    
player_shots = fetch_button.on_click(get_visuals)